In [129]:
import numpy as np
import matplotlib.pyplot as plt


def initialize_particles_center_bias(n, radius, center_bias=0.3):
    """ Initialize particles with a bias towards the center. """
    angles = np.random.uniform(0, 2 * np.pi, n)
    radii = radius * np.sqrt(np.random.uniform(0, center_bias, n))  # Bias towards smaller radii
    x = radii * np.cos(angles)
    y = radii * np.sin(angles)
    return np.column_stack((x, y))

def compute_forces_and_energy_vectorized(particles, radius):
    """ Compute the forces and energy for the current configuration of particles using vectorized operations. """
    n = len(particles)
    diff = particles[:, np.newaxis, :] - particles[np.newaxis, :, :]
    distances = np.linalg.norm(diff, axis=2)
    np.fill_diagonal(distances, np.inf)
    energy = np.sum(1 / distances[np.triu_indices(n, 1)])
    forces = np.sum(diff / distances[:,:,np.newaxis]**2, axis=1)
    return forces, energy

def update_particle_positions(particles, step_size, radius):
    """ Update the positions of particles randomly and handle infinitely hard wall boundary conditions. """
    n = len(particles)
    new_particles = np.copy(particles)

    for i in range(n):
        # Generate a random direction
        angle = np.random.uniform(0, 2 * np.pi)
        direction = np.array([np.cos(angle), np.sin(angle)])

        # Update the particle position in the random direction
        new_particles[i] += direction * step_size * np.random.uniform(0, 1)

        # Check if the particle is outside the boundary
        distance_from_center = np.linalg.norm(new_particles[i])
        if distance_from_center > radius:
            # If outside, reposition it on the boundary
            new_particles[i] = new_particles[i] / distance_from_center * radius

    return new_particles

def simulate_charged_particles_annealing(n, radius, steps, initial_step_size, final_step_size, initial_temp, final_temp):
    """ Simulate the charged particles with simulated annealing, updating all particles at once. """
    particles = initialize_particles(n, radius)
    min_energy_config = None
    min_energy = np.inf
    energies = []

    temperature = initial_temp
    step_size = initial_step_size

    # Calculate the cooling and step size rates
    cooling_rate = 0.0001 ** (1/steps)
    step_rate = (final_step_size / initial_step_size) ** (1 / steps)

    for step in range(steps):

        forces, current_energy = compute_forces_and_energy_vectorized(particles, radius)

        # Calculate new positions for all particles
        new_particles = update_particle_positions(particles, step_size, radius)
        new_energy = compute_forces_and_energy_vectorized(new_particles, radius)[1]

        # Decide whether to accept the new configuration
        if new_energy < current_energy or np.random.rand() < np.exp((current_energy - new_energy) / temperature):
            particles = new_particles
            current_energy = new_energy
        print(step, np.exp((current_energy - new_energy) / temperature), temperature)
        # Update energy, temperature, and step size after each step
        energy = compute_forces_and_energy_vectorized(particles, radius)[1]
        energies.append(energy)

        if energy < min_energy:
            min_energy = energy
            min_energy_config = particles.copy()

       
        temperature = initial_temperature * (cooling_rate ** step)
        step_size *= step_rate

    return min_energy_config, min_energy, energies

In [132]:

# Parameters
n_particles = 23  # Number of particles
circle_radius = 1  # Radius of the circle
simulation_steps = 50_000  # Number of steps in the simulation
initial_step_size = 0.3
final_step_size = 0.01
initial_temperature = 150  # Reduced initial temperature
final_temperature = 0.01  # Reduced final temperature


# Run the simulation with simulated annealing
annealing_particles, annealing_min_energy, annealing_energy_over_time = simulate_charged_particles_annealing(
    n_particles, circle_radius, simulation_steps, initial_step_size, final_step_size, initial_temperature, final_temperature
)

# Plotting
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Configuration with minimum energy (annealing)
circle = plt.Circle((0, 0), circle_radius, color='r', fill=False)
ax[0].add_artist(circle)
ax[0].scatter(annealing_particles[:, 0], annealing_particles[:, 1], color='blue')
ax[0].set_xlim([-circle_radius, circle_radius])
ax[0].set_ylim([-circle_radius, circle_radius])
ax[0].set_title("Configuration with Minimum Energy (Annealing)")
ax[0].set_aspect('equal', 'box')

# Energy over time (annealing)
ax[1].plot(annealing_energy_over_time)
ax[1].set_title("Energy Over Time (Annealing)")
ax[1].set_xlabel("Step")
ax[1].set_ylabel("Total Energy")
ax[1].axhline(y=annealing_min_energy, color='r', linestyle='--', label=f'Minimum Energy: {annealing_min_energy:.4f}')
ax[1].legend()

plt.tight_layout()
plt.show()

0 1.0 150
1 1.0 150.0
2 0.7331334770088971 149.9723715236389
3 1.0 149.94474813616253
4 1.0 149.9171298366335
5 1.0 149.88951662411475
6 1.0 149.86190849766922
7 1.0 149.8343054563602
8 1.0 149.806707499251
9 1.0 149.77911462540513
10 0.5470462387364432 149.75152683388637
11 1.0 149.7239441237586
12 1.0 149.69636649408582
13 1.0 149.66879394393234
14 1.0 149.64122647236252
15 1.0 149.61366407844093
16 0.812268569180747 149.58610676123234
17 1.0 149.55855451980167
18 1.0 149.531007353214
19 1.0 149.5034652605346
20 1.0 149.47592824082892
21 1.0 149.44839629316255
22 1.0 149.4208694166013
23 1.0 149.3933476102111
24 1.0 149.36583087305806
25 1.0 149.33831920420852
26 1.0 149.3108126027289
27 1.0 149.2833110676859
28 1.0 149.25581459814632
29 1.0 149.22832319317712
30 1.0 149.20083685184545
31 1.0 149.17335557321866
32 1.0 149.14587935636428
33 1.0 149.11840820034993
34 1.0 149.09094210424348
35 0.8114183329721238 149.06348106711295
36 1.0 149.0360250880265
37 1.0 149.0085741660525
38 1.0